## Docker Spark setup

This notebook is meant to run on spark 2 docker container. First i'll describe the steps to set it up

On Linux based system install Docker and docker-compose.Create this file : docker-compose.yml. Run docker-compose build . Afterwards : docker-compose build -d
Contents of the file is listed below


```
version: "2"

services:
  master:
    image: singularities/spark
    command: start-spark master
    hostname: master
    ports:
      - "6066:6066"
      - "7070:7070"
      - "8080:8080"
      - "50070:50070"
      - "8888:8888"
  worker:
    image: singularities/spark
    command: start-spark worker master
    environment:
      SPARK_WORKER_CORES: 1
      SPARK_WORKER_MEMORY: 2g
    links:
      - master
```

```
With docker ps , check if the master and worker containers are running.
Connect to the master node:
docker exec -it [container id master] bash
On the master node prompt continue with setup as described below.
```

## Spark and conda env setup
Install new conda env for park, using python 3.5 (3.6 has bug). Install latest Anaconda first. Also Spark 2.1 has to be installed with $SPARK_HOME set

conda create -n spark python=3.5
source active spark
source activate spark
conda install notebook ipykernel
ipython kernel install --user --name spark --display-name spark

Make jupyter start script:
PYSPARK_PYTHON=/root/anaconda3/envs/spark/bin/python
PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS='notebook --ip=0.0.0.0 --port=8888' $SPARK_HOME/bin/pyspark



In [1]:
#Start this in spark conda env to test


from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf


## Example data

This example works if you clone https://github.com/PacktPublishing/Learning-PySpark

and make sure its in /root/learningPySpark
TODO: Install git on docker image

In [1]:
flights = "file:/root/learningPySpark/Chapter03/flight-data/departuredelays.csv" 
airports = "file:/root/learningPySpark/Chapter03/flight-data/airport-codes-na.txt" 
airports_df = spark.read.csv(airports,header='true',inferSchema='true',sep='\t')
airports_df.createOrReplaceTempView("airports")
flights_df = spark.read.csv(flights,header='true')
flights_df.createOrReplaceTempView("flights")
flights_df.cache()

DataFrame[date: string, delay: string, distance: string, origin: string, destination: string]